# Point Cloud Data Visualization

* Visualize the point cloud from a
    * local array
    * cloud array
    * dictionary
* Customize with optional parameters

In [ ]:
import requests
import json

import pandas as pd
import pdal
from pybabylonjs import Show as show
import tiledb

## Optional: create a sparse TileDB array from a LAZ file

In [ ]:
!wget -nc "https://github.com/PDAL/data/blob/master/autzen/autzen-classified.laz?raw=true" -O "autzen-classified.laz"

In [ ]:
pipeline = (
  pdal.Reader("autzen-classified.laz") |
  pdal.Filter.stats() |
  pdal.Writer.tiledb(array_name="autzen-classified",chunk_size=100000)
)

count = pipeline.execute()  

## Data sources

The point cloud data can be loaded in three different ways that are specfied by the `source` parameter:
* a locally stored TileDB array (`source = "local"`)
* a TileDB Cloud array (`source = "cloud"`)
* a dictionary (`source = "dict"`)

To load and display a slice of the data a bounding box with the minimum and maximum values of X, Y and Z are needed:

In [ ]:
bbox = {
    'X': [636800, 637200],
    'Y': [852800, 853100],
    'Z': [406.14, 615.26]
}

bbox2 = {
    'X': [636800, 637800],
    'Y': [851000, 853000],
    'Z': [406.14, 615.26]
}

### Cloud array
To display point cloud data from a TileDB cloud array a TileDB `token` is needed: 
* [Sign up for a free TileDB account](https://cloud.tiledb.com/auth/signup)
* [Create a token as described here](https://docs.tiledb.com/cloud/how-to/account/create-api-tokens)
* Paste the token into the below cell (`<token>`)

In [ ]:
token = "<token>"

In [ ]:
show.point_cloud(source="cloud",
                 uri = "tiledb://TileDB-Inc/autzen_classified_tiledb",
                 token=token,
                 bbox = bbox,
                 particle_size = 6,
                 width = 1000,
                 height = 1000,
                 rgb_max = 65535,
                 camera_radius = 1000,
                 z_scale = 1.2)

### Local array
Point cloud data in a local array is loaded and displayed with the below.

In [ ]:
lidar_array = "autzen-classified"

In [ ]:
show.point_cloud(source="local",
                 uri=lidar_array,
                 bbox = bbox2,
                 particle_size = 4,
                 width = 1000,
                 height = 800,
                 rgb_max = 65535,
                 camera_radius = 1000,
                 particle_budget = 8000000,
                 zscale = 3)

### Data from a dictionary
Alternatively data can be loaded into a dictionary first and then displayed. 

Load the data directly into a dictionary from the local array:

In [ ]:
with tiledb.open(lidar_array) as arr:
    data = arr.query(attrs=["Red", "Green", "Blue"], dims=["X", "Y", "Z"])[
        bbox["X"][0] : bbox["X"][1],
        bbox["Y"][0] : bbox["Y"][1],
        bbox["Z"][0] : bbox["Z"][1],
    ]

Or first load the data into a pandas DataFrame when for example pre-processing of the data is needed:

In [ ]:
with tiledb.open(lidar_array) as arr:
    df = pd.DataFrame(arr[
        bbox["X"][0] : bbox["X"][1],
        bbox["Y"][0] : bbox["Y"][1],
        bbox["Z"][0] : bbox["Z"][1]])

In [ ]:
df = df.drop(['ReturnNumber', 'NumberOfReturns', 'ScanDirectionFlag', 'EdgeOfFlightLine', 'ScanAngleRank', 'UserData', 'PointSourceId', 'ScanChannel', 'ClassFlags'], axis=1)

In [ ]:
df

In [ ]:
data = {
    'X': df['X'],
    'Y': df['Y'],
    'Z': df['Z'],
    'Red': df['Red'],
    'Green': df['Green'],
    'Blue': df['Blue']
}

In [ ]:
show.point_cloud(source="dict",
                 data=data,
                 bbox = bbox,
                 particle_size = 2.5,
                 width = 1000,
                 height = 700,
                 rgb_max = 65535,
                 camera_radius = 700,
                 edl_strength = 3)

## Optional parameters

There are several parameters to change the way the visualization looks:

* `width` and `height` define the size of the display canvas (defaults are 800 and 600)
* `particle_size` sets the size of the points (default is 1)
* `color_scheme` sets the background color scheme, choose from "dark" (default), "light" and "blue"
* `rgb_max` is the maximum RGB value used to scale all values between 0 and 1, when not provided it will be calculated from the data
* 'camera_radius' 
* `show_fraction` can be used to display a fraction of the points, showing every *nth* point from the loaded data

An example:

In [ ]:
show.point_cloud(source="dict",
                data=data,
                bbox=bbox,
                width=800,
                height=700,
                particle_size=3,
                color_scheme="light",
                rgb_max=65535, 
                camera_radius = 1000,
                show_fraction = 4)